In [42]:
using PlotlyJS
using XLSX
include("../model/utils.jl")
include("../model/unit_commitment.jl")
include("../model/economic_dispatch.jl")
include("./plotting.jl")
include("./processing.jl")

parquet_to_solution (generic function with 1 method)

In [43]:
folder_path = joinpath("..","output", "solutions_v1.2")
# solution_folders = ["n_15--350"] 
# solution_folders = ["n_15-45-75-106-136-167-197-228-259-289-320-350"] 
# solution_folders = ["n_15", "n_45", "n_75", "n_106", "n_136", "n_167","n_197", "n_228", "n_259"]
solution_folders = ["n_136", "n_167","n_197", "n_228", "n_259"]

keys = [:demand, :generation, :storage, :reserve, :energy_reserve, :scalar]
s_uc = [parquet_to_solution("s_uc", joinpath(folder_path, s)) for s in solution_folders]
s_ed = [parquet_to_solution("s_ed", joinpath(folder_path, s)) for s in solution_folders]
s_uc = NamedTuple(k => vcat([s[k] for s in s_uc]...) for k in keys)
s_ed = NamedTuple(k => vcat([s[k] for s in s_ed]...) for k in keys)


reading...
../output/solutions_v1.2/n_136/s_uc_demand.parquet
../output/solutions_v1.2/n_136/s_uc_generation.parquet
../output/solutions_v1.2/n_136/s_uc_storage.parquet
../output/solutions_v1.2/n_136/s_uc_reserve.parquet
../output/solutions_v1.2/n_136/s_uc_energy_reserve.parquet
../output/solutions_v1.2/n_136/s_uc_scalar.parquet
...done
reading...
../output/solutions_v1.2/n_167/s_uc_demand.parquet
../output/solutions_v1.2/n_167/s_uc_generation.parquet
../output/solutions_v1.2/n_167/s_uc_storage.parquet
../output/solutions_v1.2/n_167/s_uc_reserve.parquet
../output/solutions_v1.2/n_167/s_uc_energy_reserve.parquet
../output/solutions_v1.2/n_167/s_uc_scalar.parquet
...done
reading...
../output/solutions_v1.2/n_197/s_uc_demand.parquet
../output/solutions_v1.2/n_197/s_uc_generation.parquet
../output/solutions_v1.2/n_197/s_uc_storage.parquet
../output/solutions_v1.2/n_197/s_uc_reserve.parquet
../output/solutions_v1.2/n_197/s_uc_energy_reserve.parquet
../output/solutions_v1.2/n_197/s_uc_scalar

(demand = 72000×8 DataFrame
   Row │ hour   demand_MW  r_id     resource  LOL_MW    iteration  day    conf ⋯
       │ Int64  Float64    Missing  String    Float64?  Symbol     Int64  Symb ⋯
───────┼────────────────────────────────────────────────────────────────────────
     1 │  3265    2197.39  missing  total          0.0  demand_86    136  base ⋯
     2 │  3266    1835.26  missing  total          0.0  demand_86    136  base
     3 │  3267    1621.28  missing  total          0.0  demand_86    136  base
     4 │  3268    1931.39  missing  total          0.0  demand_86    136  base
     5 │  3269    1784.23  missing  total          0.0  demand_86    136  base ⋯
     6 │  3270    2086.18  missing  total          0.0  demand_86    136  base
     7 │  3271    2482.13  missing  total          0.0  demand_86    136  base
     8 │  3272    2633.63  missing  total          0.0  demand_86    136  base
     9 │  3273    2991.6   missing  total          0.0  demand_86    136  base ⋯
    10 │  32

In [44]:
thres =.001 # 1 Watt
f_LOL(x,y) = 
    (LLD_h=count(x.>thres),
    # LOLP=count(x.>thres)/length(y)*100,
    ENS_MWh = sum(x),
    # LOL_percentage = sum(x)/(sum(y) + sum(x))*100,
    Demand_MWh = (sum(y) + sum(x)),
    )
f_CUR(x,y) =     
    (CURD_h=count(x.>thres),
    # CURP=count(x.>thres)/length(y)*100,
    CUR_MWh = sum(x),
    # CUR_percentage = sum(x)/(sum(y) + sum(x))*100,
    RES_production_MWh = (sum(y) + sum(x)),
    )

f_CUR (generic function with 1 method)

In [45]:
write = false
group_by = [:configuration, :day]
filter = in(["onshore_wind_turbine","small_hydroelectric","solar_photovoltaic", "net_generation"]).(s_ed.generation.resource)

gdf_LOL = combine(groupby(s_ed.demand, union([:iteration], group_by)), [:LOL_MW, :demand_MW] => ((x,y)->f_LOL(x,y)) => AsTable)
gdf_CUR = combine(groupby(s_ed.generation[filter,:], union([:iteration], group_by)), [:curtailment_MW, :production_MW] =>((x,y) -> f_CUR(x,y))=> AsTable)
# gdf_SCALAR = combine(groupby(s_ed.scalar, union([:iteration], group_by)), [:objective_value] => (x -> (objective_value = sum(x))) => AsTable)
gdf_LOL_CUR = outerjoin(gdf_LOL, gdf_CUR, on = union([:iteration], group_by))

LOL = combine(groupby(gdf_LOL, group_by), [:LLD_h, :ENS_MWh] => ((x,y)->(LOLE = mean(x), EENS = mean(y))) => AsTable)
CUR = combine(groupby(gdf_CUR, group_by), [:CURD_h, :CUR_MWh] => ((x,y)->(CURE = mean(x), ECUR = mean(y))) => AsTable)
SCALAR = combine(groupby(s_ed.scalar, group_by), :objective_value => mean => :EOV)
LOL_CUR = outerjoin(LOL, CUR, SCALAR, on=[:configuration, :day])

if write
    XLSX.writetable(joinpath(folder_path,"LOL_CUR.xlsx"), "LOL_CUR" => change_type(LOL_CUR, Symbol, string))
    XLSX.writetable(joinpath(folder_path,"gdf_LOL_CUR.xlsx"), "gdf_LOL_CUR" =>  change_type(gdf_LOL_CUR, Symbol, string))

end
;


In [101]:
right = rename(s_uc.scalar[:, Not(:termination_status)], :objective_value =>:objective_value_uc)
left = s_ed.scalar[:, Not(:termination_status)]
s_ed_scalar = leftjoin(left, right, on = setdiff(propertynames(left), [:objective_value,:iteration]))
leftjoin!(s_ed_scalar, gdf_LOL_CUR[!,union([:ENS_MWh,:CUR_MWh],[:configuration,:day, :iteration])], on = [:configuration,:day, :iteration])

# realtive difference with respect to s_uc
s_ed_scalar.delta_objective_value = s_ed_scalar.objective_value .- s_ed_scalar.objective_value_uc
s_ed_scalar.delta_objective_value_relative = (s_ed_scalar.objective_value .- s_ed_scalar.objective_value_uc)./s_ed_scalar.objective_value_uc


# relative difference with respect to ref_configuration = :base_ramp_storage_envelopes_up_0_dn_0
ref_configuration = :base_ramp_storage_envelopes_up_0_dn_0
ref_configuration = s_ed.scalar[s_ed.scalar.configuration .==ref_configuration,[:objective_value,:iteration,:day]]
rename!(ref_configuration, :objective_value => :objective_value_ref_conf)
leftjoin!(s_ed_scalar, ref_configuration, on = [:day, :iteration])

s_ed_scalar.delta_objective_value_ref_conf = s_ed_scalar.objective_value .- s_ed_scalar.objective_value_ref_conf
s_ed_scalar.delta_objective_value_relative_ref_conf = (s_ed_scalar.objective_value .- s_ed_scalar.objective_value_ref_conf)./s_ed_scalar.objective_value_ref_conf

if write
    XLSX.writetable(joinpath(folder_path,"s_uc_scalar.xlsx"), "s_uc_scalar" => change_type(s_uc.scalar, Symbol, string))
    XLSX.writetable(joinpath(folder_path,"s_ed_scalar.xlsx"), "s_ed_scalar" => change_type(s_ed_scalar, Symbol, string))
end


In [104]:
# leftjoin!(s_ed_scalar, s_ed[!,union([:ENS_MWh,:CUR_MWh],[:configuration,:day, :iteration])], on = [:configuration,:day, :iteration])

In [103]:
# plot(s_ed_scalar, y = :objective_value_relative, x = :ENS_MWh, mode = "markers", marker=attr(showscale=true, coloraxis="coloraxis", color=:configuration))
plot(s_ed_scalar, y = :delta_objective_value_relative, x = :ENS_MWh, mode = "markers", facet_col = :day, color= :configuration)#marker=attr(showscale=true, coloraxis="coloraxis", color=:configuration))

data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, legend, margin, template, xaxis, xaxis2, xaxis3, xaxis4, xaxis5, yaxis, yaxis2, yaxis3, yaxis4, and yaxis5"

In [150]:
plot(s_ed_scalar, y = :delta_objective_value_relative_ref_conf, x = :ENS_MWh, mode = "markers", facet_col = :day, color= :configuration)

data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, legend, margin, template, xaxis, xaxis2, xaxis3, xaxis4, xaxis5, yaxis, yaxis2, yaxis3, yaxis4, and yaxis5"

In [165]:
to_plot = s_ed_scalar[(s_ed_scalar.day .== 259) .& (abs.(s_ed_scalar.delta_objective_value_relative_ref_conf) .>10),:]
plot(to_plot, y = :delta_objective_value_relative_ref_conf, x = :iteration, facet_col= :configuration, facet_col_wrap = 3, mode = "markers", marker=attr(showscale=true, coloraxis="coloraxis", color=:ENS_MWh))

data: [
  "scatter with fields marker, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields marker, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields marker, mode, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, legend, margin, template, xaxis, xaxis2, xaxis3, yaxis, yaxis2, and yaxis3"

In [132]:
plot(gdf_LOL_CUR, x = :CUR_MWh, y = :ENS_MWh, facet_col = :configuration, text = :iteration, color= :day, mode="markers")

data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, legend, margin, template, xaxis, xaxis2, xaxis3, xaxis4, xaxis5, xaxis6, yaxis, yaxis2, yaxis3, yaxis4, yaxis5, and yaxis6"

In [50]:
# plot(LOL_CUR, x = :LOLE, y = :EENS, facet_col = :day, group = :configuration, marker_color = :EOV_M, mode = "markers", marker=attr(showscale=true))
# configs
plot(LOL_CUR, x = :ECUR, y = :EENS, facet_col = :day, mode = "markers", text = :day, color=:configuration) # marker=attr(showscale=true, coloraxis="coloraxis", color=:day

data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, legend, margin, template, xaxis, xaxis2, xaxis3, xaxis4, xaxis5, yaxis, yaxis2, yaxis3, yaxis4, and yaxis5"

In [157]:
# plot(LOL_CUR, x = :EOV, y = :EENS, text = :configuration, color = :configuration, mode = "markers")
plot(LOL_CUR[LOL_CUR.day .== 259, :], x = :EOV, y = :EENS, color = :configuration, mode = "markers")


data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields legend, margin, template, xaxis, and yaxis"

In [52]:
# plot(gdf_CUR, y = :CURP, x = :CUR_MWh, facet_col = :configuration, facet_row = :day, color= :iteration, mode="markers")

In [53]:
# plot(gdf_LOL, x = :LOL_percentage, kind="histogram", facet_col = :configuration, facet_row = :day, histonorm = "percent")

In [54]:
# filter = in(["onshore_wind_turbine","small_hydroelectric","solar_photovoltaic"]).(s_ed.generation.resource) # probably not needed if we discard the CUR_percentage KPI
# plot(gdf_CUR, x = :CUR_percentage, kind="histogram", facet_col = :configuration, facet_row = :day, histonorm = "percent")

In [55]:
supply_uc, demand_uc = calculate_supply_demand(s_uc, union([:hour, :resource], group_by))
supply_ed, demand_ed = calculate_supply_demand(s_ed, union([:hour, :resource, :iteration], group_by))
;

In [158]:
# day_ = first(s_ed.generation.day)
# day_ = 197
day_ = 259
# iteration_ = first(s_ed.generation.iteration)
# iteration_ = :demand_39
iteration_ = :demand_27



:demand_27

In [159]:
config_ = :base_ramp_storage_envelopes_up_0_dn_0
supply_uc_ = supply_uc[(supply_uc.configuration .== config_) .& (supply_uc.day .== day_), :]
demand_uc_ = demand_uc[(demand_uc.configuration .== config_) .& (demand_uc.day .== day_), :]
plot_supply_demand(supply_uc_, demand_uc_, string(config_))

data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [166]:
config_ = :base_ramp_storage_envelopes_up_1_dn_1
supply_uc_ = supply_uc[(supply_uc.configuration .== config_) .& (supply_uc.day .== day_), :]
demand_uc_ = demand_uc[(demand_uc.configuration .== config_) .& (demand_uc.day .== day_), :]
plot_supply_demand(supply_uc_, demand_uc_, string(config_))

data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [160]:
config_ = :base_ramp_storage_energy_reserve_cumulated
supply_ed_ = supply_ed[(supply_ed.configuration .== config_) .& (supply_ed.day .== day_) .& (supply_ed.iteration .== iteration_), :]
demand_ed_ = demand_ed[(demand_ed.configuration .== config_) .& (demand_ed.day .== day_) .& (demand_ed.iteration .== iteration_), :]
plot_supply_demand(supply_ed_, demand_ed_, string(config_))

data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [161]:
config_ = :base_ramp_storage_envelopes_up_1_dn_1
supply_ed_ = supply_ed[(supply_ed.configuration .== config_) .& (supply_ed.day .== day_) .& (supply_ed.iteration .== iteration_), :]
demand_ed_ = demand_ed[(demand_ed.configuration .== config_) .& (demand_ed.day .== day_) .& (demand_ed.iteration .== iteration_), :]
plot_supply_demand(supply_ed_, demand_ed_, string(config_))


data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [162]:
config_ = :base_ramp_storage_envelopes_up_0_75_dn_0_75
supply_ed_ = supply_ed[(supply_ed.configuration .== config_) .& (supply_ed.day .== day_) .& (supply_ed.iteration .== iteration_), :]
demand_ed_ = demand_ed[(demand_ed.configuration .== config_) .& (demand_ed.day .== day_) .& (demand_ed.iteration .== iteration_), :]
plot_supply_demand(supply_ed_, demand_ed_, string(config_))


data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [163]:
config_ = :base_ramp_storage_envelopes_up_0_5_dn_0_5
supply_ed_ = supply_ed[(supply_ed.configuration .== config_) .& (supply_ed.day .== day_) .& (supply_ed.iteration .== iteration_), :]
demand_ed_ = demand_ed[(demand_ed.configuration .== config_) .& (demand_ed.day .== day_) .& (demand_ed.iteration .== iteration_), :]
plot_supply_demand(supply_ed_, demand_ed_, string(config_))

data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [164]:
config_ = :base_ramp_storage_envelopes_up_0_dn_0
supply_ed_ = supply_ed[(supply_ed.configuration .== config_) .& (supply_ed.day .== day_) .& (supply_ed.iteration .== iteration_), :]
demand_ed_ = demand_ed[(demand_ed.configuration .== config_) .& (demand_ed.day .== day_) .& (demand_ed.iteration .== iteration_), :]
plot_supply_demand(supply_ed_, demand_ed_, string(config_))

data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

### Other

In [61]:
plot_reserve(reserve_, "Storage + SOC Imprudent")

LoadError: UndefVarError: `reserve_` not defined

In [ ]:
supply_ = supply_uc[supply_uc.configuration .== :base_ramp_storage_envelopes, :]
demand_ = demand_uc[demand_uc.configuration .== :base_ramp_storage_envelopes, :]
solution_reserve_ = s_uc.reserve[s_uc.reserve.configuration .== :base_ramp_storage_envelopes, :]
reserve_ = calculate_reserve(solution_reserve_, required_reserve)
plot_supply_demand(supply_, demand_,"Storage + SOC Envelopes")

In [ ]:
plot_reserve(reserve_, "Storage + SOC Envelopes")

In [ ]:
supply_ = supply_uc[supply_uc.configuration .== :base_ramp_storage_energy_reserve_cumulated, :]
demand_ = demand_uc[demand_uc.configuration .== :base_ramp_storage_energy_reserve_cumulated, :]
solution_reserve_ = s_uc.reserve[s_uc.reserve.configuration .== :base_ramp_storage_energy_reserve_cumulated, :]
reserve_ = calculate_reserve(solution_reserve_, required_reserve)
plot_supply_demand(supply_, demand_,"Storage + Energy Reserve")

In [ ]:
plot_reserve(reserve_, "Storage + Energy Reserve")

## Other

In [ ]:
demand_ = :demand_28
supply_ed_ = supply_ed[(supply_ed.configuration .== :base_ramp_storage_envelopes).&(supply_ed.iteration .== demand_), :]
demand_ed_ = demand_ed[(demand_ed.configuration .== :base_ramp_storage_envelopes).&(demand_ed.iteration .== demand_), :]
# solution_reserve_ = s_uc.reserve[s_uc.reserve.configuration .== :base_ramp_storage_envelopes, :]
# reserve_ = calculate_reserve(solution_reserve_, required_reserve)
plot_supply_demand(supply_ed_, demand_ed_,"Storage + Envelopes")
# p1 = plot_fieldx_by_fieldy(supply_ed_, :production_MW, :resource, "ED - Storage + SOC Envelopes")

# p2 = plot_fieldx_by_fieldy(supply_ed_, :production_MW, :resource, "ED - Storage + Energy Reserve")

# [p1 p2]

In [ ]:
supply_ed_ = supply_ed[(supply_ed.configuration .== :base_ramp_storage_energy_reserve_cumulated).&(supply_ed.iteration .== demand_), :]
demand_ed_ = demand_ed[(demand_ed.configuration .== :base_ramp_storage_energy_reserve_cumulated).&(demand_ed.iteration .== demand_), :]
plot_supply_demand(supply_ed_, demand_ed_,"ED - Storage + Energy Reserve")

In [ ]:
get_day = 1
days = unique(s_ed.generation.day)
gen_df, loads_multi_df, gen_variable_multi_df, storage_df, random_loads_multi_df = generate_input_data(days[get_day], "../input/base_case")
required_reserve, required_energy_reserve, required_energy_reserve_cumulated = generate_reserves(loads_multi_df, gen_variable_multi_df, 0.1)
;

In [ ]:
select_first = 100
to_plot_demand = stack(random_loads_multi_df[!,1:(select_first+1)], Not(:hour))
to_plot_reserve = copy(required_reserve)
to_plot_reserve.reserve_up_MW = required_reserve.reserve_up_MW .+ random_loads_multi_df[!,:demand]
to_plot_reserve.reserve_down_MW = -required_reserve.reserve_down_MW .+ random_loads_multi_df[!,:demand]
to_plot_reserve.demand_MW = random_loads_multi_df[!,:demand]
to_plot_reserve = stack(to_plot_reserve, Not(:hour))
s= scatter(to_plot_demand, x = :hour, y = :value, group = :variable)
s1 = scatter(to_plot_reserve, x = :hour, y = :value, group = :variable, line=attr(color="purple", width=1.5, dash="dot"))
union!(s,s1)
p1 = plot(s, Layout(yaxis_title="power MW", xaxis_title="hour"))

t_ = loads_multi_df[loads_multi_df.demand .== maximum(loads_multi_df.demand),:hour]
to_plot_histogram = stack(random_loads_multi_df[random_loads_multi_df.hour .== t_, :])
p2 = plot(to_plot_histogram, x = :value, kind="histogram", histonorm = "percent", Layout(xaxis_title_text="demand at t = $t_"))

[p1 p2]